In [44]:
from pyqubo import Qbit, Array, solve_qubo
from collections import defaultdict

In [2]:
car_routes = {
    # {車番号}: [{第一候補のルート番号}, {第二候補のルート番号}, ...]
    0: [0, 1, 2],
	1: [0, 1, 2]
}

routes = {
  # {ルート番号}: [{1つめの経由するエッジ名}, {2つめの経由するエッジ名}, ..]
	0: ["s0", "s3", "s6", "s9"],
	1: ["s0", "s3", "s8", "s11"],
	2: ["s2", "s7", "s10", "s11"]
}

In [32]:
n_cars = len(car_routes)
n_routes = len(car_routes[0])
print("n_cars", n_cars, "n_routes", n_routes)

n_cars 2 n_routes 3


In [7]:
# {エッジ名: {(車番号, ルート番号), ...}, ...}
edge_to_indices = defaultdict(set)

for car, rs in car_routes.items():
    for route in rs:
        edges = routes[route]
        for edge in edges:
            index = (car, route)
            edge_to_indices[edge].add(index)

In [8]:
edge_to_indices

defaultdict(set,
            {'s0': {(0, 0), (0, 1), (1, 0), (1, 1)},
             's3': {(0, 0), (0, 1), (1, 0), (1, 1)},
             's6': {(0, 0), (1, 0)},
             's9': {(0, 0), (1, 0)},
             's8': {(0, 1), (1, 1)},
             's11': {(0, 1), (0, 2), (1, 1), (1, 2)},
             's2': {(0, 2), (1, 2)},
             's7': {(0, 2), (1, 2)},
             's10': {(0, 2), (1, 2)}})

In [59]:
x = Array.create("x", shape=(n_cars, n_routes), vartype="BINARY")
cost = 0.0
for edge, indices in edge_to_indices.items():
    #a = [0~10の整数]
    a  = 0 #フォルクスワーゲンの場合はa=0
    c = (sum(x[i, j] for (i, j) in indices) - a)**2
    cost += c

In [40]:
# 内包表記でsum
sum(i for i in range(10))**2

2025

In [52]:
# 一個ルートを選択する制約
const = 0
for i in range(n_cars):
    const += (sum(x[i, j] for j in range(n_routes)) - 1)**2

In [54]:
param = 10.0 #制約の強さ。要調整。
model = (cost + param*const).compile()
qubo, offset = model.to_qubo()

In [55]:
qubo

{('x[0][1]', 'x[0][1]'): -6.0,
 ('x[1][0]', 'x[1][0]'): -6.0,
 ('x[0][0]', 'x[0][0]'): -6.0,
 ('x[1][1]', 'x[1][1]'): -6.0,
 ('x[0][2]', 'x[0][2]'): -6.0,
 ('x[1][2]', 'x[1][2]'): -6.0,
 ('x[0][1]', 'x[1][0]'): 4.0,
 ('x[0][0]', 'x[0][1]'): 24.0,
 ('x[0][1]', 'x[1][1]'): 8.0,
 ('x[0][0]', 'x[1][0]'): 8.0,
 ('x[1][0]', 'x[1][1]'): 24.0,
 ('x[0][0]', 'x[1][1]'): 4.0,
 ('x[0][1]', 'x[0][2]'): 22.0,
 ('x[0][1]', 'x[1][2]'): 2.0,
 ('x[0][2]', 'x[1][1]'): 2.0,
 ('x[1][1]', 'x[1][2]'): 22.0,
 ('x[0][2]', 'x[1][2]'): 8.0,
 ('x[0][0]', 'x[0][2]'): 20.0,
 ('x[1][0]', 'x[1][2]'): 20.0}

In [56]:
solve_qubo(qubo)

{'x[0][0]': 1,
 'x[0][1]': 0,
 'x[0][2]': 0,
 'x[1][0]': 0,
 'x[1][1]': 0,
 'x[1][2]': 1}